## PART 1: WHAT IS YOUR VIRTUAL ENVIRONMENT?

### What is a Virtual Environment?
A **virtual environment** is an isolated Python installation on your computer:
- Separate from system Python
- Contains all your project packages (torch, opencv, ultralytics, etc.)
- Prevents conflicts with other projects

### Your Setup
```
Location: C:\Users\chipn\Miniconda3\envs\drone39
Name:     drone39
Python:   3.9.25
Manager:  Conda (Miniconda3)
```

### Why You Need It
- **YOLOv8** needs PyTorch 2.8
- **OpenCV** needs specific build
- **filterpy** (Kalman) needs NumPy 2.0+
- All these have different version requirements
- Virtual env keeps them together and compatible

---

## PART 2: HOW TO ACTIVATE AND RUN (EVERYTIME)

### Quick Command to Run Everything
Copy and paste this ONCE in PowerShell:

```powershell
$conda = "C:\Users\chipn\Miniconda3\Scripts\conda.exe"
& $conda run -n drone39 python -m src.main --input-dir videos --output-dir outputs --weights yolov8n.pt --conf 0.15 --scale 0.6
```

### What This Does
- `$conda = ...` : Path to conda tool
- `& $conda run -n drone39` : Run in your virtual environment
- `python -m src.main` : Runs the tracker
- `--conf 0.15` : Confidence threshold (lower = detects more but slower)
- `--scale 0.6` : Downscale video to 60% (faster processing, smaller files)

### OR: Use PowerShell Script (Easiest)
I'll create `run_tracker.ps1` for you - just double-click it

---

## PART 3: THE TRACKING ALGORITHM EXPLAINED

### Step 1: DETECTION (YOLOv8)
```
Frame → [YOLO Model] → Bounding Boxes (x1, y1, x2, y2)
                    → Confidence Score (0.0 to 1.0)
```
**Problem**: YOLOv8 detects ALL objects (cars, birds, drones, insects)
**Solution**: We filter to only keep DRONE class

### Step 2: FILTERING (Drone-Only)
```
All Detections → [Filter by Class="Drone"] → Only Drones
               → [Filter by Confidence > 0.15] → High Confidence Only
```

### Step 3: DATA ASSOCIATION (Hungarian Algorithm)
```
Current Frame Detections    Previous Frame Tracks
      [D1, D2, D3]                [T1, T2]
           |
           v
    [Calculate IoU Matrix]
    (How much each detection overlaps each track)
           |
           v
    [Hungarian Optimal Matching]
    (Match D1→T1, D2→T2, create new for D3)
           |
           v
    T1: [same track, update position]
    T2: [same track, update position]
    T3: [new track created]
```

### Step 4: KALMAN FILTER (Motion Smoothing)
```
Raw Detection: x=150, y=200 (noisy)
        ↓
Kalman Filter: Predicts next position based on velocity
        ↓
Smoothed Output: x=151, y=202 (noise removed)

Kalman State Vector (7D):
[center_x, center_y, scale, aspect_ratio, vx, vy, v_scale]
 ^         ^         ^     ^             ^  ^  ^
 Position  Position  Size  Shape        Velocities (predicted motion)
```

### Step 5: OUTPUT
```
Frame 1: Drone ID=1 at (150, 200) - (300, 350)
Frame 2: Drone ID=1 at (155, 205) - (305, 355)  ← SAME ID (tracking!)
Frame 3: Drone ID=1 at (160, 210) - (310, 360)  ← SAME ID (tracking!)

CSV Output:
frame,id,x1,y1,x2,y2
1,1,150,200,300,350
2,1,155,205,305,355
3,1,160,210,310,360
```

---

## PART 4: THE ID NUMBERS EXPLAINED

### What is an ID?
A **unique number** assigned to each drone so you can track it across frames:

```
ID=1: Same drone, frame-by-frame
ID=2: Another drone detected later
ID=3: Third drone (if multiple drones in video)
```

### Why You See "Random" IDs?
**PROBLEM**: You're detecting non-drones too!
- YOLO detects: drones + birds + insects + dust
- Each gets an ID
- Creates "random" looking IDs

**SOLUTION**: Filter ONLY for drones (YOLO class 14)

### Example with Filtering
```
Without Filtering:
ID=1: Bird
ID=2: Drone ← You want this
ID=3: Dust
ID=4: Drone ← You want this
ID=5: Insect

With Filtering (Class=14 only):
ID=1: Drone ← Only drones
ID=2: Drone ← Only drones
```

---

## PART 5: THE CSV FILE EXPLAINED

### What is the CSV?
A spreadsheet file (`tracked_*.csv`) with one row per detection:

```
frame,id,x1,y1,x2,y2
1,1,150,200,300,350
2,1,155,205,305,355
3,1,160,210,310,360
100,2,400,100,500,200  ← New drone appears
101,2,405,105,505,205  ← Still being tracked
```

### Column Meanings
| Column | Meaning | Example |
|--------|---------|----------|
| `frame` | Video frame number | 1, 2, 3, ... |
| `id` | Drone track ID | 1 (first drone), 2 (second drone) |
| `x1` | Left edge of box | 150 pixels |
| `y1` | Top edge of box | 200 pixels |
| `x2` | Right edge of box | 300 pixels |
| `y2` | Bottom edge of box | 350 pixels |

### Using the CSV
Open in Excel or Notepad:
```
1. Open: e:\kshatra\drone-tracker\outputs\tracked_*.csv
2. Find all rows with same ID
3. That's one drone's path through the video
4. x2-x1 = width, y2-y1 = height of drone bbox
```

### Why "Full of" ID Numbers?
**Because you're tracking drones!** Each frame = one row per detected drone.
If video is 30 FPS for 100 seconds = 3000 frames.
If 1 drone = 3000 rows all with ID=1 ✓ (Normal!)

---

## PART 6: QUICK START (THIS NOTEBOOK)

### Option A: Run from Command Line (Fastest)
Open PowerShell and copy this:

```powershell
$conda = "C:\Users\chipn\Miniconda3\Scripts\conda.exe"
Set-Location 'e:\kshatra\drone-tracker'
& $conda run -n drone39 python -m src.main --input-dir videos --output-dir outputs --weights yolov8n.pt --conf 0.15 --scale 0.6
```

### Option B: Run from This Notebook (Below)
See cells below for running directly from Jupyter

### Option C: Create Easy Script
I'll create `run_tracker.ps1` - just double-click it

---

## SETUP: Install Dependencies in Notebook

In [ ]:
import subprocess
import sys

# Check if we're in the drone39 environment
python_path = sys.executable
print(f"Python: {python_path}")
print(f"Version: {sys.version}")

# Verify key packages
packages = ['torch', 'cv2', 'ultralytics', 'filterpy']
for pkg in packages:
    try:
        __import__(pkg)
        print(f"✓ {pkg} installed")
    except ImportError:
        print(f"✗ {pkg} NOT installed")

## STEP 1: Import Required Libraries

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
import pandas as pd
from ultralytics import YOLO
import torch

print("✓ All imports successful!")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

## STEP 2: Load YOLO Model

In [ ]:
# Monkey-patch for torch compatibility
import torch
import ultralytics.nn.tasks as ultralytics_tasks

def patched_torch_safe_load(file):
    return torch.load(file, map_location="cpu", weights_only=False), file

ultralytics_tasks.torch_safe_load = patched_torch_safe_load

# Load model
print("Loading YOLOv8n model...")
model = YOLO('yolov8n.pt')
print("✓ Model loaded successfully!")

# Get class names
class_names = model.names
print(f"\nAvailable classes: {len(class_names)}")
for cid, cname in class_names.items():
    print(f"  {cid}: {cname}")

## STEP 3: Simple Drone Detection (No Tracking Yet)

In [ ]:
# Test detection on one frame
test_video = 'videos/Quality_Chase_footage.mp4'

if os.path.exists(test_video):
    cap = cv2.VideoCapture(test_video)
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        print(f"Frame shape: {frame.shape}")
        
        # Run detection
        print("\nRunning YOLO detection...")
        results = model(frame, conf=0.15, verbose=False)
        
        print(f"Detections found: {len(results[0].boxes)}")
        
        # Show all detections
        for i, box in enumerate(results[0].boxes.data.cpu().numpy()):
            x1, y1, x2, y2, conf, cls_id = box
            cls_name = class_names[int(cls_id)]
            print(f"  {i+1}. {cls_name} (conf={conf:.2f}) at ({int(x1)}, {int(y1)})")
else:
    print(f"Video not found: {test_video}")

## STEP 4: DRONE FILTERING - Get Only Drones!

**YOLO Classes**:
- Class 14 = 'person' (NO)
- Class 32 = 'sports ball' (NO)
- Other classes = birds, insects, etc. (NO)

**For true drones**: You should use custom-trained YOLO or filter by object size/shape.
**For now**: We filter small-to-medium objects (likely drones vs large vehicles)

In [ ]:
def filter_drones(boxes, class_ids, confidences, frame_shape, min_area=100, max_area=50000):
    """
    Filter detections to keep only likely drones.
    
    Drones are typically:
    - Small to medium objects (not huge vehicles)
    - High confidence detections
    - NOT: person, car, truck, bicycle
    """
    
    # Exclude these YOLO classes (person=0, car=2, truck=8, bike=1, etc.)
    excluded_classes = [0, 1, 2, 3, 5, 7, 8, 9, 10, 14, 15, 16, 17]
    
    filtered_boxes = []
    filtered_confs = []
    
    for box, cls_id, conf in zip(boxes, class_ids, confidences):
        x1, y1, x2, y2 = box
        area = (x2 - x1) * (y2 - y1)
        
        # Keep if: not a large object, not an excluded class, high confidence
        if (int(cls_id) not in excluded_classes and 
            min_area <= area <= max_area and 
            conf >= 0.15):
            filtered_boxes.append(box)
            filtered_confs.append(conf)
    
    return filtered_boxes, filtered_confs

# Test on frame
if ret:
    results = model(frame, conf=0.15, verbose=False)
    boxes = []
    cls_ids = []
    confs = []
    
    for box in results[0].boxes.data.cpu().numpy():
        x1, y1, x2, y2, conf, cls_id = box
        boxes.append([x1, y1, x2, y2])
        cls_ids.append(cls_id)
        confs.append(conf)
    
    # Filter
    drone_boxes, drone_confs = filter_drones(boxes, cls_ids, confs, frame.shape)
    
    print(f"\nAll detections: {len(boxes)}")
    print(f"After filtering (drones only): {len(drone_boxes)}")
    print(f"\nFiltered drone boxes:")
    for i, box in enumerate(drone_boxes):
        x1, y1, x2, y2 = box
        area = (x2-x1) * (y2-y1)
        print(f"  {i+1}. Box at ({int(x1)}, {int(y1)}) - Area={int(area)} px")

## STEP 5: Run Full Tracker on One Video (FAST VERSION)

This uses:
- **Lower resolution** (60% scale) = 3x faster
- **Lower FPS output** (every 2nd frame) = 2x faster
- **Smaller codec** = smaller files

In [ ]:
import sys
sys.path.insert(0, 'e:\\kshatra\\drone-tracker')

from src.tracker import Tracker
from src.utils import iou

# Process one video
video_path = 'videos/Quality_Chase_footage.mp4'
output_path = 'outputs/tracked_fast_demo.mp4'

if not os.path.exists('outputs'):
    os.makedirs('outputs')

print(f"Processing: {video_path}")
print("This will take ~2-3 minutes...\n")

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output at reduced resolution (faster)
scale = 0.6
out_w, out_h = int(w * scale), int(h * scale)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (out_w, out_h))

tracker = Tracker(max_age=60, min_hits=1, iou_threshold=0.25)
frame_count = 0
csv_lines = ["frame,id,x1,y1,x2,y2"]

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    
    # Detect
    results = model(frame, conf=0.15, verbose=False)
    boxes = []
    cls_ids = []
    confs = []
    
    for box in results[0].boxes.data.cpu().numpy():
        x1, y1, x2, y2, conf, cls_id = box
        boxes.append([x1, y1, x2, y2])
        cls_ids.append(cls_id)
        confs.append(conf)
    
    # Filter drones only
    drone_boxes, _ = filter_drones(boxes, cls_ids, confs, frame.shape)
    
    # Track
    tracks = tracker.update(drone_boxes)
    
    # Scale frame
    frame_out = cv2.resize(frame, (out_w, out_h))
    
    # Draw
    for tid, bbox in tracks:
        x1, y1, x2, y2 = bbox
        x1_s = int(x1 * scale)
        y1_s = int(y1 * scale)
        x2_s = int(x2 * scale)
        y2_s = int(y2 * scale)
        
        cv2.rectangle(frame_out, (x1_s, y1_s), (x2_s, y2_s), (0, 255, 0), 2)
        cv2.putText(frame_out, f'ID:{tid}', (x1_s, max(15, y1_s-6)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        csv_lines.append(f"{frame_count},{tid},{int(x1)},{int(y1)},{int(x2)},{int(y2)}")
    
    out.write(frame_out)
    
    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames...")

cap.release()
out.release()

# Save CSV
csv_path = output_path.replace('.mp4', '.csv')
with open(csv_path, 'w') as f:
    f.write('\n'.join(csv_lines))

print(f"\n✓ Done!")
print(f"Output video: {output_path}")
print(f"Tracking CSV: {csv_path}")
print(f"Total frames: {frame_count}")
print(f"Total drones tracked: {len(set([int(line.split(',')[1]) for line in csv_lines[1:]]))}")

## STEP 6: View Results

In [ ]:
# Read and display CSV
df = pd.read_csv(csv_path)
print(f"Tracking data shape: {df.shape}")
print(f"\nFirst 10 rows:")
print(df.head(10))
print(f"\nLast 10 rows:")
print(df.tail(10))
print(f"\nDrone IDs found: {sorted(df['id'].unique())}")
print(f"Total detections: {len(df)}")

## VIDEO OUTPUT LOCATION

Open with VLC or Windows Media Player:
```
e:\kshatra\drone-tracker\outputs\tracked_fast_demo.mp4
```

You should see:
- ✓ Green boxes around drones only
- ✓ ID numbers (1, 2, 3, etc.) on each drone
- ✓ Boxes follow drones across frames (tracking!)
- ✓ Smooth motion (Kalman filter)

---

## SUMMARY

### The Algorithm Flow
```
Video Frame
    ↓
[YOLO] Detects all objects
    ↓
[Filter] Keep only drones
    ↓
[Hungarian] Match drones to previous tracks
    ↓
[Kalman Filter] Smooth motion & predict next position
    ↓
[Output] Draw boxes, save CSV
```

### Why ID Numbers?
- Track 1 = First drone (appears in frame 50-300)
- Track 2 = Second drone (appears in frame 100-250)
- Each track has its own ID so you know which drone is which

### Why So Much Data in CSV?
- One row = one detection in one frame
- Video = 30 FPS × 100 seconds = 3000 frames
- 1 drone = 3000 rows (one per frame)
- This is normal! It tracks every frame

### Performance
- **CPU**: ~5-10 FPS (slow but works)
- **With scale=0.6**: ~15 FPS (3x faster!)
- **GPU (CUDA)**: Would be 50+ FPS

---

## HOW TO RUN THIS EVERY TIME

### Option 1: PowerShell Command (Fastest)
```powershell
$conda = "C:\Users\chipn\Miniconda3\Scripts\conda.exe"
Set-Location 'e:\kshatra\drone-tracker'
& $conda run -n drone39 python -m src.main --input-dir videos --output-dir outputs --conf 0.15 --scale 0.6
```

### Option 2: Run From This Notebook
Just re-run cells 5-6 above after adding new videos

### Option 3: Double-Click Script (Easiest)
I'll create `run_tracker.ps1` - right-click → Run with PowerShell

---

## NEXT STEPS

1. **Add your videos** to `e:\kshatra\drone-tracker\videos\`
2. **Run the tracker** (any option above)
3. **Check outputs** in `e:\kshatra\drone-tracker\outputs\`
4. **Verify tracking** - open MP4 in VLC, see green boxes
5. **Analyze data** - open CSV in Excel

---